In [2]:
from multiprocessing import Process, Pipe
import numpy as np

In [22]:
class A():
    def __init__(self, name):
        self.name = name
        self.last_proposal = np.random.rand()
        
    def start_task(self):
        return self.last_proposal
    
    def propose(self, conn):
        print("Starting Agent {}".format(self.name))
        agr = conn.recv()
        while agr != 'stop':
            self.last_proposal = self.last_proposal + 0.05*(agr-self.last_proposal)
            conn.send(self.last_proposal)
            agr = conn.recv()
        print("Agent {} stopped".format(self.name))
        
        
class M():
    def __init__(self):
        self.agents = [A('ag_'+str(n)) for n in range(4)]
        self.conns = [Pipe() for a in self.agents]
        self.processes = [Process(target=self.agents[a].propose, args=(self.conns[a][1],)) for a in range(len(self.agents))]
    
    def negotiate(self):
        proposals = np.array([a.start_task() for a in self.agents])
        for ag, (parent_conn, child_conn), ag_proc in zip(self.agents, self.conns, self.processes):
                ag_proc.start()
                
        for i in range(100):
            agreement = np.mean(proposals)
            for ag, (parent_conn, child_conn), ag_proc in zip(self.agents, self.conns, self.processes):
                parent_conn.send(agreement)

            proposals = list()
            for ag, (parent_conn, child_conn), ag_proc in zip(self.agents, self.conns, self.processes):
                proposals.append(parent_conn.recv())

            print('\r {} {} {} {}'.format(proposals[0], proposals[1], proposals[2], proposals[3]))
            proposals = np.array(proposals)
        
        for ag, (parent_conn, child_conn), ag_proc in zip(self.agents, self.conns, self.processes):
            parent_conn.send('stop')
            ag_proc.join()
        
        
mod = M()
mod.negotiate()

Starting Agent ag_0
Starting Agent ag_2
Starting Agent ag_1
Starting Agent ag_3
Agent ag_0 stopped
Agent ag_1 stopped
Agent ag_2 stopped
Agent ag_3 stopped
 0.2347929450792541 0.20378236052935475 0.920849791348304 0.30023326650758736
 0.24379902736859765 0.21433897204619326 0.8955530313241951 0.30596733272551424
 0.252354805543474 0.22436775298718986 0.8715211093012916 0.31141469563254476
 0.26048279480960657 0.23389509488113663 0.8486907833795332 0.3165896903942238
 0.2682043846124325 0.24294606968038607 0.8270019737538628 0.32150593541781886
 0.27553989492511716 0.251544495739673 0.8063976046094758 0.32617636819023416
 0.28250862972216756 0.2597130004959956 0.7868234539223083 0.3306132793240287
 0.28912892777936544 0.2674730800145021 0.7682280107694992 0.3348283449011335
 0.2954182109337034 0.2748451555570832 0.7505623397743305 0.3388326571993831
 0.30139302993032446 0.2818486273225353 0.7337799523289202 0.3426367538827202
 0.3070691079771145 0.28850192549971476 0.7178366842557805 0.

In [23]:
mod.negotiate()

AssertionError: cannot start a process twice